In [14]:
import pandas as pd
import urllib.request
import os
from datetime import datetime,date
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import json
import yaml
from calendar import monthrange
import xarray as xr
import cdsapi
import zipfile
from glob import glob
#important
#before running this script, please go to cads-forms-json, git pull, then git checkout prod


# wishlist
# - put land hydrology just below cryosphere
# - update repository cds-forms-c3s and change to c3sprod
#   - git checkout c3sprod
#   - git pull
# - download datasets that only contain info in the time dimension from CDS
# - try to make the code more path-independent
# - install on athos and run every week. update some figure repo


def extract_dates_from_TSI():
    url='https://gerb.oma.be/tsi/C3S_RMIB_daily_TSI_composite_ICDR_v3_latest.txt'
    c=pd.read_csv(url,skiprows=128,delim_whitespace=True,header=None)
    return pd.Timestamp(str(c[3].iloc[0])),pd.Timestamp(str(c[3].iloc[-1]))
def extract_dates_icesheets(datasets_dir):
    # opens TCDR and ICDR files for both Antarctica and Greenland, then computes max/min dates
    # needs adjustment for per product sorting
    fnames = glob(datasets_dir+'C3S_*IS_RA*.nc')
    print(fnames)
    datebegs = []
    dateends = []
    for fname in fnames:
        print('will open '+fname)
        nc = xr.open_dataset(fname)
        datebegs.append(pd.Timestamp(nc['time'].values[0]))
        dateends.append(pd.Timestamp(nc['time'].values[-1]))
    return min(datebegs),max(dateends)
def extract_dates_massbalance(datasets_dir):
    fname = datasets_dir+'C3S_GMB_GRACE_vers4.nc'
    nc = xr.open_dataset(fname)
    return pd.Timestamp(nc['time'].values[0]),pd.Timestamp(nc['time'].values[-1])
def extract_dates_derived_glaciers(jfile):
    with open(jfile) as f:
        gen= yaml.safe_load(f)[0]
    ymin = int(gen['hydrological_year'][0][0:4])
    ymax = int(gen['hydrological_year'][-1][0:4])+1
    mmax=9
    dmax=30
    mmin=4
    dmin=1
    print('Glaciers ',ymin,ymax)
    return pd.Timestamp(f'{ymin}-{mmin}-{dmin}'), pd.Timestamp(f'{ymax}-{mmax}-{dmax}')
def extract_dates_lake_levels(datasets_dir):
    # need better way of estimating this
    # for v4, according to overview pages, coverage goes until 2022
    # need to implement download of all lakes data and check their time coverage
    print('...for lake water levels, only reading info for Lake Volta...')
    fname = datasets_dir+'C3S_LWL_N-AFRICA_VOLTA_HYLA0000156_07.45N_000.11E_altimetry_5.0_19921013_20231224_R20240109.nc'
    nc = xr.open_dataset(fname)
    return pd.Timestamp(nc['time'].values[0]),pd.Timestamp(nc['time'].values[-1])
def datemax2(row):
    row = row.dropna()
    ymax = int(max(row['year']))
    if 'month' in row.keys():
        mmax = int(max(row['month']))
    else:
        mmax=12
    xx,dmax=monthrange(ymax,mmax)
    datemax = pd.Timestamp(f'{ymax}-{mmax}-{dmax}')
    return datemax
def datemin2(row):
    row = row.dropna()
    ymin = int(min(row['year']))
    if 'month' in row.keys():
        mmin = int(min(row['month']))
    else:
        mmin=1
    xx,dmin=monthrange(ymin,mmin)
    datemin = pd.Timestamp(f'{ymin}-{mmin}-{dmin}')
    return datemin
def extract_dates_wv(jfilepath):
    f = open(jfilepath)
    # returns JSON object as 
    # a dictionary
    data = json.load(f)
    # print(data)
    df = pd.DataFrame(data)
    df['datemax'] = df.apply(datemax2,axis=1)
    df['datemin'] = df.apply(datemin2,axis=1)
    datemin = df['datemin'].min()
    datemax = df['datemax'].max()
    return datemin,datemax 
def check_time_agg(row):
    row = row.dropna()
    if 'time_aggregation' in row.keys():
        time_aggregation = row['time_aggregation'][0]
        # print('time_agg',time_aggregation)
        if time_aggregation in ['daily_average','daily_mean','day','day_average']: 
            time_agg = 'day'
            return time_agg
        elif time_aggregation == 'daily':
            time_agg='daily'
            return time_agg
        #note interim solution for 5-daily-composite...
        elif time_aggregation in [
            'monthly_average',
            '5_daily_composite',
            'monthly_mean',
            '27_days',
            'month',
            'monthly',
            'month_average',
            '10_day_average', # debatable..
            ]:
            time_agg = 'monthly'
            return time_agg
        else:
            # print(row)
            print('Could not determine time_agg')
            raise SystemExit
    elif 'period' in row.keys(): # applies to ice_sheets
        time_agg = 'period'
        return time_agg
    elif 'temporal_aggregation' in row.keys():
        time_aggregation = row['temporal_aggregation'][0]
        if time_aggregation in ['monthly','6-hourly']:
            time_agg='monthly'
        elif time_aggregation == 'daily':
            time_agg='daily'
        else:
            print('Error in temporal aggregation')
            raise SystemExit
        return time_agg
    else:
        if ('day' in row.keys()):
            time_agg='day'
            return time_agg
        elif  'nominal_day' in row.keys():
            time_agg='nominal_day'
            return time_agg
        else:
            time_agg = 'monthly'
            return time_agg
def compute_datemax(row):
 
    time_agg=check_time_agg(row) # check time aggregation of data in this row
    # print('time_agg',time_agg)
    if time_agg =='period':
        per_str=max(row['period'])
        ymax=int(per_str[5::])
        mmax=9
        dmax=30
    else: 
        ymax = int(max(row['year']))
        if 'month' in row.keys():
            mmax = int(max(row['month']))
        else:
            mmax=12
        xx,ndays = monthrange(ymax,mmax)
        if time_agg in ['day','nominal_day']:
            # print(row[time_agg])
            dmax = int(max(row[time_agg])) 
        elif time_agg in ['daily']:
            dmax = int(max(row['day'])) 
        else:
            dmax = ndays # last day of month
        if dmax >ndays:
            print('Beware error in allowed dates...')
            # print(row)
            dmax=ndays

    datemax = pd.Timestamp(f'{ymax}-{mmax}-{dmax}')
    return datemax
def compute_datemin(row):
    time_agg=check_time_agg(row) # check time aggregation of data in this row
    if time_agg =='period':
        per_str=max(row['period'])
        ymin=int(per_str[0:4])
        mmin=10
        dmin=1
    else:
        ymin = int(min(row['year'])) 
        if 'month' in row.keys():
            mmin = int(min(row['month']))
        else:
            mmin=1
        if time_agg in ['day','nominal_day']:
            dmin = int(min(row[time_agg])) 
        elif time_agg in ['daily']:
            dmin = int(min(row['day'])) 
        else:
            dmin = 1 # first day of month
        xx,ndays = monthrange(ymin,mmin)     
        if dmin>ndays:
            print('Beware error in allowed dates...')
            # print(row)
            dmin=1
    datemin = pd.Timestamp(f'{ymin}-{mmin}-{dmin}')
    return datemin
def calc_dateminmax_from_cds_form(jfilepath,ecv):
    # Opening JSON file
    f = open(jfilepath)
    # returns JSON object as 
    # a dictionary
    data = json.load(f)
    # print(data)
    df = pd.DataFrame(data)
    # display(df)
    # print(df.keys())
    # print(len(df))
    
    # find records where dates cannot be defined
    if 'sensor_and_algorithm' in df.keys():        
        lst_erase=[]
        for i in range(len(df)):
            if (df['sensor_and_algorithm'][i][0]=='merged_obs4mips'): lst_erase.append(i)
        # now .drop these problematic rows
        for i in lst_erase:
            df=df.drop(lst_erase)
    if ecv == 'Earth Radiation Budget':        
        lst_erase=[]
        for i in range(len(df)):
            if (df['variable'][i][0]=='total_solar_irradiance'): lst_erase.append(i) # this info is read from the dataset itself
        # now .drop these problematic rows
        for i in lst_erase:
            df=df.drop(lst_erase)
    # for i in range(len(df)):
    #     print(df.loc[i])
    #     if ('year' not in df[i]): lst_erase.append(i)

    df['datemax'] = df.apply(compute_datemax,axis=1)
    df['datemin'] = df.apply(compute_datemin,axis=1)
    datemin = df['datemin'].min()
    datemax = df['datemax'].max()
    return datemin,datemax
def datasets_download(datasets_dir):
    c = cdsapi.Client()
    c.retrieve(
        'satellite-ice-sheet-mass-balance',
        {
            'variable': 'all',
            'format': 'zip',
        },
        datasets_dir+'download.zip')
    os.system(f'cd {datasets_dir} ; unzip download.zip; rm -rf download.zip ; cd ..')
    
    # Antarctica Ice sheet surface elevation change TCDR
    c.retrieve(
    'satellite-ice-sheet-elevation-change',
    {
        'domain': 'antarctica',
        'climate_data_record_type': 'tcdr',
        'version': '5_0',
        'variable': 'all',
        'format': 'zip',
    },
    datasets_dir+'download.zip')
    os.system(f'cd {datasets_dir} ; unzip download.zip; rm -rf download.zip ; cd ..')
    
    # Antarctica Ice sheet surface elevation change ICDR
    c.retrieve(
        'satellite-ice-sheet-elevation-change',
    {
        'domain': 'antarctica',
        'climate_data_record_type': 'icdr',
        'version': '3_0',
        'variable': 'all',
        'format': 'zip',
    },
    datasets_dir+'download.zip')
    os.system(f'cd {datasets_dir} ; unzip download.zip; rm -rf download.zip ; cd ..')
    
    
    # Greenland Ice sheet surface elevation change TCDR
    c.retrieve(
        'satellite-ice-sheet-elevation-change',
    {
        'domain': 'greenland',
        'climate_data_record_type': 'tcdr',
        'version': '4_0',
        'variable': 'all',
        'format': 'zip',
    },
    datasets_dir+'download.zip')
    os.system(f'cd {datasets_dir} ; unzip download.zip; rm -rf download.zip ; cd ..')
    
    # Greenland Ice sheet surface elevation change ICDR
    c.retrieve(
        'satellite-ice-sheet-elevation-change',
    {
        'domain': 'greenland',
        'climate_data_record_type': 'icdr',
        'version': '4_0',
        'variable': 'all',
        'format': 'zip',
    },
    datasets_dir+'download.zip')
    os.system(f'cd {datasets_dir} ; unzip download.zip; rm -rf download.zip ; cd ..')
    
    # Lake water levels
    
    c.retrieve(
        'satellite-lake-water-level',
    {
        'variable': 'all',
        'version': 'version_4_0',
        'format': 'zip',
        'region': 'northern_africa',
        'lake': 'volta',
    },
    datasets_dir+'download.zip')
    os.system(f'cd {datasets_dir} ; unzip download.zip; rm -rf download.zip ; cd ..')
      
    
    
    return
with open('config-athos.yml') as f:
    conf= yaml.safe_load(f)

cds_form_dir=conf['cds_form_dir']
datasets_dir = conf['datasets_dir']

# datasets_download(datasets_dir) # uncomment to update datasets downloaded from CDS



### BUILD PANDAS DATAFRAME FOR TIME COVERAGE BY ECV 

In [15]:

datesbeg = {}
datesend = {}
ecv_dic = {}
for k,ecv in enumerate(conf['ECV']):
    print(ecv)
    entries = conf['ECV'][ecv]['entry']
    print(entries)
    datemin_list = []
    datemax_list = []
    if ecv in ['Earth Radiation Budget']: 
        datemin,datemax = extract_dates_from_TSI()
        datemin_list.append(datemin)
        datemax_list.append(datemax)
    for entry in entries:
        jfilepath=f'{cds_form_dir}{entry}/constraints.json'
        print(jfilepath)
        if entry == 'satellite-ice-sheet-elevation-change':
            datemin,datemax = extract_dates_icesheets(datasets_dir)
            datemin_list.append(datemin)
            datemax_list.append(datemax)
        elif entry == 'satellite-ice-sheet-mass-balance':
            datemin,datemax = extract_dates_massbalance(datasets_dir)
            datemin_list.append(datemin)
            datemax_list.append(datemax)
        elif entry == 'derived-gridded-glacier-mass-change':
            jfile = f'{cds_form_dir}{entry}/constraints.json'
            datemin,datemax = extract_dates_derived_glaciers(jfile)
            datemin_list.append(datemin)
            datemax_list.append(datemax)
        elif entry == 'insitu-glaciers-extent':    
            datemin= pd.Timestamp('1990-01-01') # http://www.glims.org/rgi_user_guide/06_dataset_summary.html
            datemax= pd.Timestamp('2010-12-31')
            datemin_list.append(datemin)
            datemax_list.append(datemax)
        elif entry =='satellite-total-column-water-vapour-ocean':
            # temporal aggregation is messed up. does not have the same meaning as other datasets
            # monthly should be yearly
            # 6-hourly should be monthly
            # need to write a special function that accounts for this
            datemin,datemax = extract_dates_wv(jfilepath)
            datemin_list.append(datemin)
            datemax_list.append(datemax)
        elif entry == 'satellite-lake-water-level':
            datemin,datemax = extract_dates_lake_levels(datasets_dir)
            datemin_list.append(datemin)
            datemax_list.append(datemax)
        # elif jfilepath == '/Users/cxjo/Documents/cds-forms-c3s/satellite-land-cover/constraints.json':
        #     datemin_list.append(pd.Timestamp('1992-01-01'))
        #     datemax_list.append(pd.Timestamp('2022-12-31'))
        else:
            # print(jfilepath)
            datemin,datemax = calc_dateminmax_from_cds_form(jfilepath,ecv)
            # print(ecv,datemin_list,datemax_list)
            datemin_list.append(datemin)
            datemax_list.append(datemax)
            # print(ecv,datemin_list,datemax_list)

    # now get the max and min per ECV, accounting for all products
    datemin_list = np.array(datemin_list)
    datemax_list = np.array(datemax_list)
    # datesbeg[ecv] = np.min(datemin_list)
    # datesend[ecv] = np.max(datemax_list)
    ecv_dic[k] = {
        'ECV'     : ecv,
        'DateBeg' : np.min(datemin_list),
        'DateEnd' : np.max(datemax_list),
        'Thematic Hub' : conf['ECV'][ecv]['Thematic_hub']
    }


# ecv_pd = pd.DataFrame([conf['ECV'].keys(),datesbeg,datesend],index=['DateBeg','DateEnd']).T
ecv_pd = pd.DataFrame.from_dict(ecv_dic,orient='index').sort_values(['Thematic Hub'])
ecv_pd['DateBeg'] = ecv_pd['DateBeg'].dt.ceil(freq='s')  
ecv_pd['DateEnd'] = ecv_pd['DateEnd'].dt.ceil(freq='s')  
ecv_pd['DateEnd'] = ecv_pd['DateEnd'].apply(lambda dt: dt.strftime("%Y-%m-%d"))
ecv_pd['DateBeg'] = ecv_pd['DateBeg'].apply(lambda dt: dt.strftime("%Y-%m-%d"))

print(ecv_pd.to_markdown())
ecv_pd.to_excel('ECV_time_coverage_perECV.xlsx')
# fig = px.timeline(ecv_pd, x_start="DateBeg", x_end="DateEnd", y='Product',color='Lot')
fig = px.timeline(ecv_pd, x_start="DateBeg", x_end="DateEnd",y='ECV',color='Thematic Hub')

# fig = px.timeline(datasets_df, x_start="startdate", x_end="enddate", y='ECV')
fig.update_yaxes(autorange="reversed")
fig.update_layout(
    autosize=False,
    width=1200,
    height=800,
)
# fig.update_layout(
#     xaxis = dict(
#         dtick = 'Y1',
#         tickformat="%Y",
#     )
# )

xlab = np.arange(1970,2025).astype('int')
xlabtxt = [f'{i}' for i in xlab]


fig.update_xaxes(minor=dict(ticks="inside", showgrid=True))
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = xlab,
        ticktext = xlabtxt
    )
)
fig.update_xaxes(tickangle=-45)
fig.update_layout(
    xaxis = dict(
        tickfont = dict(
            size=10),
        )
    )
fig.update_xaxes(range = ['1970-01-01','2024-12-31'])
# print(fig)
today_date = pd.Timestamp.today().strftime('%Y%m%d') 
print(today_date)
fig.write_image(f'temporal_coverage_by_ECV_{today_date}.pdf')
fig.write_image(f'temporal_coverage_by_ECV_{today_date}.png')
fig.show()

Aerosols
['satellite-aerosol-properties']
/home/cxjo/C3S_stuff/cads-forms-json/satellite-aerosol-properties/constraints.json
Albedo
['satellite-albedo']
/home/cxjo/C3S_stuff/cads-forms-json/satellite-albedo/constraints.json
Greenhouse Gases
['satellite-carbon-dioxide', 'satellite-methane']
/home/cxjo/C3S_stuff/cads-forms-json/satellite-carbon-dioxide/constraints.json
/home/cxjo/C3S_stuff/cads-forms-json/satellite-methane/constraints.json
Clouds
['satellite-cloud-properties']
/home/cxjo/C3S_stuff/cads-forms-json/satellite-cloud-properties/constraints.json
Earth Radiation Budget
['satellite-earth-radiation-budget']


/etc/ecmwf/ssd/ssd1/tmpdirs/cxjo.3863126.20241218_113208.111/ipykernel_327573/2395731837.py:31: FutureWarning:

The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead



/home/cxjo/C3S_stuff/cads-forms-json/satellite-earth-radiation-budget/constraints.json
Fire
['satellite-fire-burned-area', 'satellite-fire-radiative-power']
/home/cxjo/C3S_stuff/cads-forms-json/satellite-fire-burned-area/constraints.json
/home/cxjo/C3S_stuff/cads-forms-json/satellite-fire-radiative-power/constraints.json
Ice Sheets
['satellite-greenland-ice-sheet-velocity', 'satellite-ice-sheet-elevation-change', 'satellite-ice-sheet-mass-balance']
/home/cxjo/C3S_stuff/cads-forms-json/satellite-greenland-ice-sheet-velocity/constraints.json
/home/cxjo/C3S_stuff/cads-forms-json/satellite-ice-sheet-elevation-change/constraints.json
['/home/cxjo/C3S_stuff/datasets/C3S_GrIS_RA_SEC_25km_vers4_2022-10-20.nc', '/home/cxjo/C3S_stuff/datasets/C3S_GrIS_RA_SEC_25km_vers5_2023-12-15.nc', '/home/cxjo/C3S_stuff/datasets/C3S_GrIS_RA_SEC_25km_vers5_2024-07-07.nc', '/home/cxjo/C3S_stuff/datasets/C3S_AntIS_RA_SEC_25km_vers4_2022-12-16.nc', '/home/cxjo/C3S_stuff/datasets/C3S_AntIS_RA_SEC_vers3_2023-01-16.

### Build pandas dataframe for the figure ordered by product


In [16]:
def calc_dateminmax_from_cds_form_2(jfilepath,ecv):
    # Opening JSON file
    f = open(jfilepath)
    # returns JSON object as 
    # a dictionary
    data = json.load(f)
    # print(data)
    df = pd.DataFrame(data)
    # display(df)
    
    # print(df.keys())
    # print(len(df))
    
    # find records where dates cannot be defined
    if 'sensor_and_algorithm' in df.keys():        
        lst_erase=[]
        for i in range(len(df)):
            if (df['sensor_and_algorithm'][i][0]=='merged_obs4mips'): lst_erase.append(i)
        # now .drop these problematic rows
        for i in lst_erase:
            df=df.drop(lst_erase)
    if ecv == 'Earth Radiation Budget':        
        lst_erase=[]
        for i in range(len(df)):
            if (df['variable'][i][0]=='total_solar_irradiance'): lst_erase.append(i) # this info is read from the dataset itself
        # now .drop these problematic rows
        for i in lst_erase:
            df=df.drop(lst_erase)
    # for i in range(len(df)):
    #     print(df.loc[i])
    #     if ('year' not in df[i]): lst_erase.append(i)

    df['datemax'] = df.apply(compute_datemax,axis=1)
    df['datemin'] = df.apply(compute_datemin,axis=1)

    datemin = df['datemin'].min()
    datemax = df['datemax'].max()

    return datemin,datemax

prod_dic = {}
datemax_list = []
datemin_list = []

for k,prod in enumerate(conf['PRODUCT']):
    # print(prod, conf['PRODUCT'][prod]['ECV'])
    ecv = conf['PRODUCT'][prod]['ECV']
    entry = conf['PRODUCT'][prod]['entry'][0]
    product = conf['PRODUCT'][prod]['Product']
    themHub = conf['PRODUCT'][prod]['Thematic_hub']
    jfilepath=f'{cds_form_dir}{entry}/constraints.json'
    print(jfilepath,prod)
    if prod == 'C3S_RMIB_ERB':
        datemin,datemax = extract_dates_from_TSI()
    elif prod == 'Ice Sheet Gravimetric Mass Balance':
        fname = glob(datasets_dir+'C3S_GMB*')[0]
        nc = xr.open_dataset(fname)
        datemin,datemax=(pd.Timestamp(nc['time'].values[0]),pd.Timestamp(nc['time'].values[-1]))
    elif prod == 'Ice Sheet Surface Elevation Change (Antarctica)':
        fnames = glob(datasets_dir+'C3S_*AntIS_RA*.nc')
        datebegs = []
        dateends = []
        for fname in fnames:
            print('will open '+fname)
            nc = xr.open_dataset(fname)
            datebegs.append(pd.Timestamp(nc['time'].values[0]))
            dateends.append(pd.Timestamp(nc['time'].values[-1]))
        datemin,datemax=(min(datebegs),max(dateends))
        # print(prod,datemin,datemax)
        # raise SystemExit
    elif prod == 'Ice Sheet Surface Elevation Change (Greenland)':
        fnames = glob(datasets_dir+'C3S_*GrIS_RA*.nc')
        datebegs = []
        dateends = []
        for fname in fnames:
            print('will open '+fname)
            nc = xr.open_dataset(fname)
            print(pd.Timestamp(nc['time'].values[0]))
            datebegs.append(pd.Timestamp(nc['time'].values[0]))
            dateends.append(pd.Timestamp(nc['time'].values[-1]))
        datemin,datemax=(min(datebegs),max(dateends))
        # print(prod,datemin,datemax)
        # raise SystemExit
    elif prod == 'Glaciers elevation and mass change data':
        jfile = f'{cds_form_dir}{entry}/constraints.json'
        print(jfile)
        datemin,datemax = extract_dates_derived_glaciers(jfile)
        datemin_list.append(datemin)
        datemax_list.append(datemax)
    elif entry == 'insitu-glaciers-extent':    
        datemin= pd.Timestamp('1990-01-01') # http://www.glims.org/rgi_user_guide/06_dataset_summary.html
        datemax= pd.Timestamp('2010-12-31')
        datemin_list.append(datemin)
        datemax_list.append(datemax)
    elif entry =='satellite-total-column-water-vapour-ocean':
        # temporal aggregation is messed up. does not have the same meaning as other datasets
        # monthly should be yearly
        # 6-hourly should be monthly
        # need to write a special function that accounts for this
        datemin,datemax = extract_dates_wv(jfilepath)
        datemin_list.append(datemin)
        datemax_list.append(datemax)
    elif entry == 'satellite-lake-water-level':
        datemin,datemax = extract_dates_lake_levels(datasets_dir)
        datemin_list.append(datemin)
        datemax_list.append(datemax)
    else:
        datemin,datemax = calc_dateminmax_from_cds_form_2(jfilepath,ecv)
    print(prod,datemin,datemax)
    datemin_list.append(datemin)
    datemax_list.append(datemax)
    
    prod_dic[k] = {
    'Product': product,
    'ECV'     : ecv,
    'DateBeg' : datemin,
    'DateEnd' : datemax,
    'Thematic Hub' : themHub
}

/home/cxjo/C3S_stuff/cads-forms-json/satellite-aerosol-properties/constraints.json Aerosols
Aerosols 1995-06-01 00:00:00 2023-10-31 00:00:00
/home/cxjo/C3S_stuff/cads-forms-json/satellite-carbon-dioxide/constraints.json CO2
CO2 2002-10-01 00:00:00 2022-12-31 00:00:00
/home/cxjo/C3S_stuff/cads-forms-json/satellite-methane/constraints.json CH4
CH4 2002-10-01 00:00:00 2022-12-31 00:00:00
/home/cxjo/C3S_stuff/cads-forms-json/satellite-ozone-v1/constraints.json Ozone
Ozone 1970-04-01 00:00:00 2023-07-31 00:00:00
/home/cxjo/C3S_stuff/cads-forms-json/satellite-cloud-properties/constraints.json Clouds
Clouds 1979-01-01 00:00:00 2023-10-31 00:00:00
/home/cxjo/C3S_stuff/cads-forms-json/satellite-earth-radiation-budget/constraints.json NASA_CERES_ERB
NASA_CERES_ERB 1979-01-01 00:00:00 2024-09-17 00:00:00
/home/cxjo/C3S_stuff/cads-forms-json/satellite-earth-radiation-budget/constraints.json NOAA_HIRS
NOAA_HIRS 1979-01-01 00:00:00 2024-09-17 00:00:00
/home/cxjo/C3S_stuff/cads-forms-json/satellite-e

/etc/ecmwf/ssd/ssd1/tmpdirs/cxjo.3863126.20241218_113208.111/ipykernel_327573/2395731837.py:31: FutureWarning:

The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead



C3S_RMIB_ERB 1979-01-01 00:00:00 2024-12-13 00:00:00
/home/cxjo/C3S_stuff/cads-forms-json/satellite-precipitation/constraints.json Precipitation
Precipitation 1979-01-01 00:00:00 2024-03-31 00:00:00
/home/cxjo/C3S_stuff/cads-forms-json/satellite-precipitation-microwave/constraints.json Precipitation_microwave
Precipitation_microwave 2000-01-01 00:00:00 2017-12-31 00:00:00
/home/cxjo/C3S_stuff/cads-forms-json/satellite-precipitation-microwave-infrared/constraints.json Precipitation_GIRAFE
Precipitation_GIRAFE 2002-01-01 00:00:00 2022-12-31 00:00:00
/home/cxjo/C3S_stuff/cads-forms-json/satellite-surface-radiation-budget/constraints.json Surface Radiation Budget (CMSAF/CLARA)
Surface Radiation Budget (CMSAF/CLARA) 1979-01-01 00:00:00 2023-10-31 00:00:00
/home/cxjo/C3S_stuff/cads-forms-json/satellite-surface-radiation-budget/constraints.json Surface Radiation Budget (CCI)
Surface Radiation Budget (CCI) 1979-01-01 00:00:00 2023-10-31 00:00:00
/home/cxjo/C3S_stuff/cads-forms-json/satellite-t

In [17]:


# ecv_pd = pd.DataFrame([conf['ECV'].keys(),datesbeg,datesend],index=['DateBeg','DateEnd']).T
prod_pd = pd.DataFrame.from_dict(prod_dic,orient='index').sort_values(['Thematic Hub','ECV','Product'])
prod_pd['DateBeg'] = prod_pd['DateBeg'].dt.ceil(freq='s')  
prod_pd['DateEnd'] = prod_pd['DateEnd'].dt.ceil(freq='s')  
prod_pd['DateEnd'] = prod_pd['DateEnd'].apply(lambda dt: dt.strftime("%Y-%m-%d"))
prod_pd['DateBeg'] = prod_pd['DateBeg'].apply(lambda dt: dt.strftime("%Y-%m-%d"))

print(prod_pd.to_markdown())
prod_pd.to_excel('ECV_time_coverage_perProduct.xlsx')

# fig = px.timeline(ecv_pd, x_start="DateBeg", x_end="DateEnd", y='Product',color='Lot')
fig = px.timeline(prod_pd, x_start="DateBeg", x_end="DateEnd",y='Product',color='Thematic Hub')

# fig = px.timeline(datasets_df, x_start="startdate", x_end="enddate", y='ECV')
fig.update_yaxes(autorange="reversed")
fig.update_layout(
    autosize=False,
    width=1200,
    height=800,
)
# fig.update_layout(
#     xaxis = dict(
#         dtick = 'Y1',
#         tickformat="%Y",
#     )
# )

xlab = np.arange(1970,2025).astype('int')
xlabtxt = [f'{i}' for i in xlab]


fig.update_xaxes(minor=dict(ticks="inside", showgrid=True))
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = xlab,
        ticktext = xlabtxt
    )
)
fig.update_xaxes(tickangle=-45)
fig.update_layout(
    xaxis = dict(
        tickfont = dict(
            size=10),
        )
    )
fig.update_xaxes(range = ['1970-01-01','2024-12-31'])
# print(fig)
today_date = pd.Timestamp.today().strftime('%Y%m%d') 
print(today_date)
fig.write_image(f'temporal_coverage_by_Product_{today_date}.pdf')
fig.write_image(f'temporal_coverage_by_Product_{today_date}.png')
fig.show()

|    | Product                                         | ECV                      | DateBeg    | DateEnd    | Thematic Hub            |
|---:|:------------------------------------------------|:-------------------------|:-----------|:-----------|:------------------------|
|  0 | Aerosols                                        | Aerosols                 | 1995-06-01 | 2023-10-31 | Atmospheric Composition |
|  2 | CH4                                             | Greenhouse Gases         | 2002-10-01 | 2022-12-31 | Atmospheric Composition |
|  1 | CO2                                             | Greenhouse Gases         | 2002-10-01 | 2022-12-31 | Atmospheric Composition |
|  3 | Ozone                                           | Ozone                    | 1970-04-01 | 2023-07-31 | Atmospheric Composition |
|  4 | Clouds                                          | Clouds                   | 1979-01-01 | 2023-10-31 | Atmospheric Physics     |
|  8 | Earth Rad Budget - C3S CCI               